In [5]:
import osmium as o
import svgwrite

class OSMHandler(o.SimpleHandler):
    def __init__(self, dwg):
        super(OSMHandler, self).__init__()
        self.dwg = dwg

    def way(self, w):
        if 'highway' in w.tags:  # Filter for streets
            line = []
            for node in w.nodes:
                if node.location.valid():
                    line.append((node.location.lon, node.location.lat))
            if line:
                self.dwg.add(self.dwg.line(line, stroke='white'))

def main(osm_file, svg_file):
    # Create an SVG drawing
    dwg = svgwrite.Drawing(svg_file, profile='tiny')
    #.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='rgb(128,128,128)'))

    # Create an instance of the OSM handler
    handler = OSMHandler(dwg)

    # Apply the handler to the OSM file
    handler.apply_file(osm_file)

    # Save the SVG file
    dwg.save()

if __name__ == "__main__":
    main("map.osm", "map.svg")


In [4]:
import osmnx as ox
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def create_svg_from_osm(place_name, svg_filename):
    # Configure OSMnx to use the white-black color scheme and no background
    ox.config(log_console=True, use_cache=True)
    color = mcolors.to_rgba('black')
    bgcolor = mcolors.to_rgba('white')

    # Retrieve the street network for the specified place
    #G = ox.graph_from_place(place_name, network_type='walk', simplify=True)
    G = ox.graph_from_bbox(north=46.5000, east=9.8446, south=46.4872, west=9.8109, network_type='walk', simplify=True)

    # Plot the street network and save as SVG
    fig, ax = ox.plot_graph(G, bgcolor=bgcolor, edge_color=color, node_size=0, 
                            edge_linewidth=2, show=False, save=False)

    # Save the figure as an SVG file
    plt.savefig(svg_filename, format='svg', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

# Example usage
create_svg_from_osm("St. Moritz, Switzerland", "stmoritz.svg")


C:\Users\johan\AppData\Local\Temp\ipykernel_18052\3207891558.py:7: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(log_console=True, use_cache=True)


In [3]:
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from shapely.geometry import LineString, MultiLineString

def create_svg_for_lasercutting(place_name, svg_filename, buffer_distance):
    # Configure OSMnx settings
    ox.config(log_console=True, use_cache=True)

    # Retrieve the street network for the specified place
    G = ox.graph_from_bbox(north=46.5000, east=9.8446, south=46.4872, west=9.8109, network_type='walk', simplify=True)

    # Convert the graph to a GeoDataFrame
    gdf_edges = ox.graph_to_gdfs(G, nodes=False)

    # Create a buffer around each edge (street)
    buffered_edges = gdf_edges.geometry.buffer(buffer_distance)

    # Combine buffered edges into a single geometry for plotting
    buffered_area = gpd.GeoSeries(buffered_edges.unary_union)

    # Plot the buffered area
    fig, ax = plt.subplots()
    buffered_area.plot(ax=ax, color='black')

    # Set the background color and remove axis for clarity
    ax.set_facecolor(mcolors.to_rgba('gray'))
    ax.axis('off')

    # Save the figure as an SVG file
    plt.savefig(svg_filename, format='svg', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

# Example usage with a small buffer distance
create_svg_for_lasercutting("St. Moritz, Switzerland", "stmoritz_lasercut.svg", buffer_distance=0.0001)


C:\Users\johan\AppData\Local\Temp\ipykernel_18052\3000327336.py:9: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(log_console=True, use_cache=True)
C:\Users\johan\AppData\Local\Temp\ipykernel_18052\3000327336.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffered_edges = gdf_edges.geometry.buffer(buffer_distance)


In [9]:
from svgpathtools import svg2paths, Path
import osmnx as ox
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cairosvg
import svgwrite

def create_svg_from_osm(place_name, svg_filename):
    # Configure OSMnx to use the white-black color scheme and no background
    ox.config(log_console=True, use_cache=True)
    color = mcolors.to_rgba('black')
    bgcolor = mcolors.to_rgba('white')

    # Retrieve the street network for the specified place
    G = ox.graph_from_place(place_name, network_type='walk', simplify=True)

    # Plot the street network and save as SVG
    fig, ax = ox.plot_graph(G, bgcolor=bgcolor, edge_color=color, node_size=0, 
                            edge_linewidth=2, show=False, save=False)

    # Save the figure as an SVG file
    plt.savefig(svg_filename, format='svg', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Convert the strokes in the SVG to paths using cairosvg
    dwg = svgwrite.Drawing(svg_filename)
    paths, attributes = svg2paths(svg_filename)
    
    # Create a new SVG with paths and save it
    new_svg_filename = svg_filename.replace(".svg", "_paths.svg")
    for i, path in enumerate(paths):
        new_path = Path(path)
        dwg.add(dwg.path(d=new_path.d(),fill="red", stroke='black'))
    
    dwg.saveas(new_svg_filename)

# Example usage
create_svg_from_osm("St. Moritz, Switzerland", "stmoritz.svg")

C:\Users\johan\AppData\Local\Temp\ipykernel_8696\4000196401.py:10: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(log_console=True, use_cache=True)


In [ ]:
svg